# 5. Benchmark `deeplake`

In [1]:
import deeplake
import os
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import IPython.display as ipd
from utils import time_me, time_me_seconds, SAMPLE_RATE
from benchmarks import run_all_benchmarks

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./secret/hpml-399816-55d1dc19c012.json"

In [3]:
def deeplake_cropper(crop_duration):
    crop_samples = int(SAMPLE_RATE * crop_duration)

    def crop(audio):
        audio = audio[:, 0]
        audio = audio / np.abs(audio).max()  # peak normalize
        audio = audio.astype(np.float32)  # convert to f32
        num_samples = audio.shape[0]
        assert num_samples != 0
        if num_samples < crop_samples:
            # pad if input is short
            audio = np.pad(audio, (0, crop_samples - num_samples))
        elif num_samples > crop_samples:
            # crop if it is too long
            rand_start = np.random.randint(0, num_samples - crop_samples)
            audio = audio[rand_start : rand_start + crop_samples]
        else:
            # just right :)
            pass
        assert audio.shape[0] == crop_samples

        return audio[None, :]

    return crop

In [4]:
@time_me_seconds
def build_deeplake_dataloader(
    deeplake_path,
    crop_duration=3.0,
    batch_size=32,
    shuffle_buffer=2048,
    num_workers=4,
    pin_memory=True,
):
    shuffle_buffer_size = (
        310 / 2048 * shuffle_buffer
    )  # deeplake uses MB as measurement for buffer
    ds = deeplake.load(deeplake_path)
    dataloader = ds.pytorch(
        tensors=["audio", "accent", "gender", "age"],
        transform={
            "audio": deeplake_cropper(crop_duration=crop_duration),
            "accent": None,
            "gender": None,
            "age": None,
        },
        num_workers=num_workers,
        batch_size=batch_size,
        pin_memory=pin_memory,
        shuffle=True,
        buffer_size=shuffle_buffer_size,
    )
    single_worker_dataloader = ds.pytorch(
        tensors=["audio", "accent", "gender", "age"],
        transform={
            "audio": deeplake_cropper(crop_duration=crop_duration),
            "accent": None,
            "gender": None,
            "age": None,
        },
        num_workers=0,
        batch_size=batch_size,
        pin_memory=pin_memory,
        shuffle=True,
        buffer_size=shuffle_buffer_size,
    )
    return dataloader, single_worker_dataloader

In [5]:
local_deeplake_path = "./data/deeplake/"
cloud_deeplake_path = "gcs://hpml-project/deeplake/"

In [6]:
tag = "deeplake_local"
dataloader, single_worker_dataloader = build_deeplake_dataloader(local_deeplake_path)
run_all_benchmarks(dataloader, single_worker_dataloader, tag=tag)


[2023-12-17_18-59-49]	
+-----------------------------------+
|Beginning build_deeplake_dataloader|
+-----------------------------------+
./data/deeplake/ loaded successfully.



[2023-12-17_18-59-55]	
+-------------------------------------------+
|build_deeplake_dataloader took 5.86 seconds|
+-------------------------------------------+



STAGE:2023-12-17 18:59:56 3422968:3422968 ActivityProfilerController.cpp:311] Completed Stage: Warm Up


Number of parameters of model: 64,628,259


  0%|                                                                                             | 0/200 [00:00<?, ?it/s]
Please wait, filling up the shuffle buffer with samples.:   0%|                                | 0.00/296M [00:00<?, ?B/s]
Please wait, filling up the shuffle buffer with samples.:   0%|                      | 188k/296M [00:06<2:45:14, 31.2kB/s]
Please wait, filling up the shuffle buffer with samples.:   8%|█▊                     | 23.6M/296M [00:06<00:51, 5.52MB/s]
Please wait, filling up the shuffle buffer with samples.:  16%|███▋                   | 47.1M/296M [00:06<00:20, 12.6MB/s]
Please wait, filling up the shuffle buffer with samples.:  24%|█████▍                 | 70.5M/296M [00:06<00:10, 21.7MB/s]
Please wait, filling up the shuffle buffer with samples.:  30%|██████▊                | 88.1M/296M [00:06<00:07, 29.9MB/s]
Please wait, filling up the shuffle buffer with samples.:  36%|████████▌               | 106M/296M [00:06<00:05, 38.4MB/s]
Please wait, fil

Shuffle buffer filling is complete.


/home/kaandonbekci/miniconda3/envs/ccu/lib/python3.11/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
100%|███████████████████████████████████████████████████████████████████████████████████| 200/200 [00:19<00:00, 10.22it/s]
STAGE:2023-12-17 19:00:15 3422968:3422968 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-12-17 19:00:15 3422968:3422968 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


Time to 1st batch: 8.84 seconds


In [7]:
tag = "deeplake_cloud"
dataloader, single_worker_dataloader = build_deeplake_dataloader(cloud_deeplake_path)
run_all_benchmarks(dataloader, single_worker_dataloader, tag=tag)


[2023-12-17_19-00-41]	
+-----------------------------------+
|Beginning build_deeplake_dataloader|
+-----------------------------------+


-

gcs://hpml-project/deeplake/ loaded successfully.



[2023-12-17_19-01-14]	
+--------------------------------------------+
|build_deeplake_dataloader took 33.76 seconds|
+--------------------------------------------+

Number of parameters of model: 64,628,259


STAGE:2023-12-17 19:01:15 3422968:3422968 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
  0%|                                                                                             | 0/200 [00:00<?, ?it/s]
Please wait, filling up the shuffle buffer with samples.:   0%|                                | 0.00/296M [00:00<?, ?B/s]
Please wait, filling up the shuffle buffer with samples.:   0%|                      | 188k/296M [00:10<4:40:42, 18.4kB/s]
Please wait, filling up the shuffle buffer with samples.:   8%|█▊                     | 23.6M/296M [00:11<01:41, 2.81MB/s]
Please wait, filling up the shuffle buffer with samples.:  20%|████▌                  | 58.8M/296M [00:12<00:29, 8.52MB/s]
Please wait, filling up the shuffle buffer with samples.:  24%|█████▍                 | 70.5M/296M [00:13<00:28, 8.26MB/s]
Please wait, filling up the shuffle buffer with samples.:  32%|███████▎               | 93.9M/296M [00:15<00:19, 10.6MB/s]
Please wait, filling up the shuffle 

Shuffle buffer filling is complete.


100%|███████████████████████████████████████████████████████████████████████████████████| 200/200 [01:01<00:00,  3.24it/s]
STAGE:2023-12-17 19:02:17 3422968:3422968 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-12-17 19:02:17 3422968:3422968 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


Time to 1st batch: 21.78 seconds
